Blah de blah

In [1]:
# This tool will not ensure that pixels and boundaries align
# That is the job of the user
import pandas
import numpy
from osgeo import gdal, gdalconst
from osgeo import ogr
import tempfile
import os
import sys
sys.path.append('../Modules')
import zonal_stats_ForLayerObject
import resample_raster
#from rasterstats import zonal_stats
import matplotlib.pyplot as plt
%matplotlib inline

#Path to intersected Subcatch/FUs shapefile
pathToShapefile = "D:/MW/spatial_RC7/ExportedCatchments-FUs Intersection.shp"
#pathToShapefile = "D:/MW/spatial_RC7/mw_intscfu_clean.shp"
#pathToShapefile = "D:/MW/spatial_RC7/badFUOnly.shp"

uniqueIDField = "UNQID"
catchmentField = "IntSCs"
FUField = "IntFUs"

FUsToInclude = ["Grazing Open","Grazing Forested","Forestry","Conservation"]

#Path to template raster (probably subcatchments or DEM)
rasterTemplateFile = "D:/MW/spatial_RC7/CreatedSubcats.asc"

#Output intersected Subcat/FU raster
#Will get a CSV of same name written 'mapping' each FU to a unique ID (the unique ID will be the raster values)
rasterOutputFile = "D:/MW/spatial_RC7/MW_USLE_IntSubCatFu.tif"

#Path to K raster
rasterKFile = "D:/MW/Ram_USLE/mw_k_fact.asc"

#Path to LS raster
rasterLSFile = "D:/MW/Ram_USLE/mw_ls_fact.asc"

#Path to Fines raster
rasterFinesFile = "D:/MW/Ram_USLE/mw_cs_surf.asc"

#Path to Scald raster
rasterScaldFile = "D:/MW/spatial_RC7/CreatedSubcats.asc"

#Directory of CFactor rasters
cFactorDir = "D:/MW/spatial_RC7"

#declare a couple of variables
IntSCFUStr = "IntSCFU"
MeanKLSCStr = "Mean_KLSC"
MeanKLSCFinesStr = "Mean_KLSC_Fines"
MeanKStr = "Mean_K"
MeanFinesStr = "Mean_Fines"
MeanCfactStr = "Mean_CFact"
MeanLSStr = "Mean_LS"
MeanScaldStr = "Mean_Scald"
AreaStr = "Area"
TimeStampStr = "Time_Stamp"
CellCountStr = "Cell_Count"

#Directory and filename constants for USLE parameterisation
USLE_DIR_KLSC = "KLSC";
USLE_DIR_KLSCFINE = "KLSC_Fines";
USLE_DIR_KLSC_CFACT = "Cfact";
USLE_FILE_KLSC = "USLE_KLSC_Total";
USLE_FILE_KLSCFINES = "USLE_KLSC_FinePerc";
USLE_FILE_KLS_CFACT = "CFactor";

rasterResultsDF = pandas.DataFrame(columns=[IntSCFUStr,TimeStampStr,MeanKLSCStr,MeanKLSCFinesStr,MeanKStr,MeanCfactStr,MeanFinesStr,MeanLSStr,MeanScaldStr,AreaStr,CellCountStr])
#rasterResultsDF

In [2]:
#load the template raster
templateRas = gdal.Open(rasterTemplateFile)
template_GeoTrans = templateRas.GetGeoTransform()
template_GeoTrans

(1677868.63051, 30.0, 0.0, -2257960.4690071, 0.0, -30.0)

In [3]:
#Load K factor
kFactRas = gdal.Open(rasterKFile)
kfact_GeoTrans = kFactRas.GetGeoTransform()
kfact_GeoTrans

(1672918.63051, 30.0, 0.0, -2251120.4690071, 0.0, -30.0)

In [4]:
if (kfact_GeoTrans == template_GeoTrans):
    print("Kfactor raster resolution info matches template, excellent")
else:
    print("Kfactor raster resolution info didn't match template, attempted a resample")
    kFactRas = resample_raster.resample_raster_to_match_template(templateRas, kFactRas, os.path.join(tempfile.gettempdir(), "resampledKFact.tif"))
    #kFactRas = resample_raster_to_match_template(templateRas, kFactRas, "D:/aa/tester.tif")
    kfact_GeoTrans = kFactRas.GetGeoTransform()

Kfactor raster resolution info didn't match template, attempted a resample


In [5]:
if (kfact_GeoTrans == template_GeoTrans):
    print("Kfactor raster resolution info now matches template, excellent")
    kFactRas = None
    kFactRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledKFact.tif"))
    kfact_GeoTrans = kFactRas.GetGeoTransform()
else:
    print("Kfactor raster resolution still not the same as the template, you shouldn't proceed")
#zone_ds = None
#print("rows: " + str(kFactRas.RasterYSize) + " cols: " + str(kFactRas.RasterXSize))

Kfactor raster resolution info now matches template, excellent


In [6]:
#kFactRas.GetRasterBand(1).GetNoDataValue()
templateRas.GetRasterBand(1).GetNoDataValue()

# The line below would get the kfactor raster written to disk, but it may not be necessary
#kFactRas = None

#this stuf to draw up a sample of KFactor, have to chop off heaps of edges so that the range in the colorbar doesn't consider -9999
#kFactRasArray = kFactRas.ReadAsArray()
#array_chop = kFactRasArray[3500:-3500,1500:-1500]  # chop off the outer specified pixels
#plt.imshow(array_chop, interpolation='nearest')
#plt.colorbar()
#plt.title('K Factor')
#plt.show()

-9999.0

In [7]:
#Load LS factor
lsFactRas = gdal.Open(rasterLSFile)
lsFact_GeoTrans = lsFactRas.GetGeoTransform()


In [8]:
if (lsFact_GeoTrans == template_GeoTrans):
    print("LSfactor raster resolution info matches template, excellent")
else:
    print("LSfactor raster resolution info didn't match template, attempted a resample")
    lsFactRas = resample_raster.resample_raster_to_match_template(templateRas, lsFactRas, os.path.join(tempfile.gettempdir(), "resampledLSFact.tif"))
    lsFact_GeoTrans = lsFactRas.GetGeoTransform()

LSfactor raster resolution info didn't match template, attempted a resample


In [9]:
if (lsFact_GeoTrans == template_GeoTrans):
    print("LSfactor raster resolution info now matches template, excellent")
    lsFactRas = None
    lsFactRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledLSFact.tif"))
    lsfact_GeoTrans = lsFactRas.GetGeoTransform()
else:
    print("LSfactor raster resolution still not the same as the template, you shouldn't proceed")

LSfactor raster resolution info now matches template, excellent


In [10]:
#Load Fines raster
FinesRas = gdal.Open(rasterFinesFile)
fines_GeoTrans = FinesRas.GetGeoTransform()

In [11]:
if (fines_GeoTrans == template_GeoTrans):
    print("Fines raster resolution info matches template, excellent")
else:
    print("Fines raster resolution info didn't match template, attempted a resample")
    FinesRas = resample_raster.resample_raster_to_match_template(templateRas, FinesRas, os.path.join(tempfile.gettempdir(), "resampledFines.tif"))
    fines_GeoTrans = FinesRas.GetGeoTransform()

Fines raster resolution info didn't match template, attempted a resample


In [12]:
if (fines_GeoTrans == template_GeoTrans):
    print("Fines raster resolution info now matches template, excellent")
    FinesRas = None
    FinesRas = gdal.Open(os.path.join(tempfile.gettempdir(), "resampledFines.tif"))
    fines_GeoTrans = FinesRas.GetGeoTransform()
else:
    print("Fines raster resolution info still not the same as the template, you shouldn't proceed")

Fines raster resolution info now matches template, excellent


In [13]:
#Open shapefile
basePolys = ogr.Open(pathToShapefile)
type(basePolys)
nlay = basePolys.GetLayerCount()
lyr = basePolys.GetLayer(0)
ext = lyr.GetExtent()
#nlay
#ext
lyr.GetFeatureCount()
#plt.imshow(lyr)

1286

In [14]:
featCount = 0
for feat in lyr:
    ##print("Doing something")
    #print(feat.GetFieldAsString(catchmentField) + " and FU " + feat.GetFieldAsString(FUField))
    featCount += 1
    if (featCount == 20):
        break

In [15]:
#FUsToInclude = ["Grazing Open","Grazing Forested","Forestry"]

FUsAsSQL = ""
count = 0
for thisFU in FUsToInclude:
    if(count == 0):
        FUsAsSQL = "'" + thisFU + "'"
    else:
        FUsAsSQL += ",'" + thisFU + "'"
    count += 1

FUsAsSQL

"'Grazing Open','Grazing Forested','Forestry','Conservation'"

In [16]:
lyr.SetAttributeFilter( FUField + " IN (" + FUsAsSQL + ")")
lyr.GetFeatureCount()

644

In [17]:
csvFile = rasterOutputFile.replace(".tif",".csv")
fumapper = pandas.DataFrame(columns=[uniqueIDField, catchmentField, FUField])
#featCount = 0
for feat in lyr:
    #print(feat.GetFieldAsString(catchmentField) + " and FU " + feat.GetFieldAsString(FUField))
    #fumapper.append([feat.GetFieldAsString(uniqueIDField),feat.GetFieldAsString(catchmentField),feat.GetFieldAsString(FUField)])
    newList = [feat.GetFieldAsString(uniqueIDField),feat.GetFieldAsString(catchmentField),feat.GetFieldAsString(FUField)]
    fumapper.loc[len(fumapper)] = newList
    #featCount += 1
    #if (featCount == 20):
    #    break

#lyr.GetFeatureCount()
#fumapper.to_csv(csvFile, index=False)
#fumapper

In [18]:
##help(templateRas)
#geo_transform = templateRas.GetGeoTransform()
#x_min = geo_transform[0]
#y_max = geo_transform[3]
#x_max = x_min + geo_transform[1] * templateRas.RasterXSize
#y_min = y_max + geo_transform[5] * templateRas.RasterYSize
#x_count = templateRas.RasterXSize
#y_count = templateRas.RasterYSize
#pixel_width = geo_transform[1]

#y_min

In [19]:
#zone_ds = gdal.GetDriverByName("GTiff").Create(rasterOutputFile, x_count, y_count, 1, gdal.GDT_Int32)
##Doing MEM did not yield the expected unique vals
##zone_ds = gdal.GetDriverByName("MEM").Create("", x_count, y_count, 1, gdal.GDT_Byte)
#zone_ds.SetGeoTransform((x_min, pixel_width, 0, y_min, 0, pixel_width))
#band = zone_ds.GetRasterBand(1)
#NoData_value = -9999
#band.SetNoDataValue(NoData_value)
#band.FlushCache()
#gdal.RasterizeLayer(zone_ds, [1], lyr, options=["ATTRIBUTE=" + uniqueIDField])

In [20]:
##This bit essential to get the raster file written - but could we use it before writing out (and then having to read back in?)
##Hopefully not needed if MEM type
##zone_ds = None
#type(zone_ds)

In [21]:
#zone_ds.GetGeoTransform()

In [22]:
#zonesRasterArray = zone_ds.ReadAsArray()
#uniq = numpy.unique(zonesRasterArray)
##print(uniq)


In [23]:
#type(lyr)
#stats = zonal_stats(lyr, kFactRas)

In [24]:
#K factor zonal stats
Kfact_stats = zonal_stats_ForLayerObject.zonal_stats(lyr, uniqueIDField, kFactRas, -9999)
# Will get 'converting a masked element to nan' error for tiny polygons

Doing zonal stats on 644 features


../Modules\zonal_stats_ForLayerObject.py:149: UserWarning: Warning: converting a masked element to nan.
  'min': float(masked.min()),
../Modules\zonal_stats_ForLayerObject.py:150: UserWarning: Warning: converting a masked element to nan.
  'mean': float(masked.mean()),
../Modules\zonal_stats_ForLayerObject.py:151: UserWarning: Warning: converting a masked element to nan.
  'max': float(masked.max()),
../Modules\zonal_stats_ForLayerObject.py:152: UserWarning: Warning: converting a masked element to nan.
  'std': float(masked.std()),
../Modules\zonal_stats_ForLayerObject.py:153: UserWarning: Warning: converting a masked element to nan.
  'sum': float(masked.sum()),


Length of stats: 644


In [25]:
#Load LS factor zonal stats
LSfact_stats = zonal_stats_ForLayerObject.zonal_stats(lyr, uniqueIDField, lsFactRas, -9999)

Doing zonal stats on 644 features


../Modules\zonal_stats_ForLayerObject.py:149: UserWarning: Warning: converting a masked element to nan.
  'min': float(masked.min()),
../Modules\zonal_stats_ForLayerObject.py:150: UserWarning: Warning: converting a masked element to nan.
  'mean': float(masked.mean()),
../Modules\zonal_stats_ForLayerObject.py:151: UserWarning: Warning: converting a masked element to nan.
  'max': float(masked.max()),
../Modules\zonal_stats_ForLayerObject.py:152: UserWarning: Warning: converting a masked element to nan.
  'std': float(masked.std()),
../Modules\zonal_stats_ForLayerObject.py:153: UserWarning: Warning: converting a masked element to nan.
  'sum': float(masked.sum()),


Length of stats: 644


In [26]:
#Fines raster zonal stats
Fines_stats = zonal_stats_ForLayerObject.zonal_stats(lyr, uniqueIDField, FinesRas, -9999)


Doing zonal stats on 644 features


../Modules\zonal_stats_ForLayerObject.py:149: UserWarning: Warning: converting a masked element to nan.
  'min': float(masked.min()),
../Modules\zonal_stats_ForLayerObject.py:150: UserWarning: Warning: converting a masked element to nan.
  'mean': float(masked.mean()),
../Modules\zonal_stats_ForLayerObject.py:151: UserWarning: Warning: converting a masked element to nan.
  'max': float(masked.max()),
../Modules\zonal_stats_ForLayerObject.py:152: UserWarning: Warning: converting a masked element to nan.
  'std': float(masked.std()),
../Modules\zonal_stats_ForLayerObject.py:153: UserWarning: Warning: converting a masked element to nan.
  'sum': float(masked.sum()),


Length of stats: 644


In [28]:
kFactRas = None
lsFactRas = None
FinesRas = None

index = 0
while index < len(Kfact_stats):
    if(Kfact_stats[index][uniqueIDField] == 1199):
        print(str(Kfact_stats[index][uniqueIDField]) + " KFactor Mean: " + str(Kfact_stats[index]['mean']) + " Count: " + str(Kfact_stats[index]['count']))
        print(str(LSfact_stats[index][uniqueIDField]) + " LS Mean: " + str(LSfact_stats[index]['mean']) + " Count: " + str(LSfact_stats[index]['count']))
        print(str(Fines_stats[index][uniqueIDField]) + " Fines Mean: " + str(Fines_stats[index]['mean']) + " Count: " + str(Fines_stats[index]['count']))
        break
    
    index += 1

1199 KFactor Mean: 0.04923342310234939 Count: 88491
1199 LS Mean: 1.197374471889364 Count: 87458
1199 Fines Mean: 42.397780565255225 Count: 88491


In [ ]:
def analyseThisCFactorRaster(theCFactorRas):
    # do pixel by pixel analysis
    # zones raster of intersected FUs will exist, and KLSFines raster will be co-located with this
    # send back a dictionary that also includes the valid/invalid cFactor numbers for each FU
    # should be able to access rasters etc without passing them into this function
    # same for the dictionary of results, it could already exist
    
    
    return